# HUGGING FACE TRANSFORMER FINE TUNING

In [1]:
# ! pip install transformers
# ! pip install datasets

In [1]:
import os
import cv2
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC
from tensorflow.keras.regularizers import L2
import sklearn
from transformers import ViTFeatureExtractor, TFViTModel
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

/opt/homebrew/Caskroom/miniforge/base/envs/HuggingFaceTransformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
wandb.init(name = "Hugging Face Transformer", project="Emotion-Detection", entity="amanjn2003-santa-clara-university", settings=wandb.Settings(init_timeout=300))

In [ ]:
wandb.config = {
    "CLASS_NAMES" : ['angry', 'happy', 'sad'],
    "BATCH_SIZE" : 32,
    "IMAGE_SIZE" : 224,
    "LEARNING_RATE" : 5e-5, # Very low LR for Fine-Tuning
    "N_EPOCHS" : 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 128,
    "N_DENSE_2" : 128,
    "NUM_CLASSES" : 3,
    "PATCH_SIZE" : 16,
}

CONFIGURATION = wandb.config

lossFunction = tf.keras.losses.CategoricalCrossentropy() # If Labels as One-Hot-Encodings
METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_k_accuracy")]

trainDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/train"
testDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test"

trainDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='training',
)

valDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='validation',
)

testDataset = tf.keras.utils.image_dataset_from_directory(
    testDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=None,
    subset=None,
)

trainDataset = trainDataset.prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.prefetch(tf.data.AUTOTUNE)

In [3]:
resizeRescaleHuggingFace = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224, interpolation='bilinear', name='resize'),
    tf.keras.layers.Rescaling(1./255, name='rescale'),
    tf.keras.layers.Permute((3, 1, 2), name='permute')
])

## Loading Base Pre-Trained Model & Adding CLassifier Layers to it

In [ ]:
baseModel= TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k") # Feature Extractor

input = tf.keras.layers.Input(shape=(224, 224, 3), name="image")
x = resizeRescaleHuggingFace(input)
x = baseModel.vit(x)[0][:, 0, :]

output = tf.keras.layers.Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax")(x)

HFModel = tf.keras.Model(input, output)

In [ ]:
testImage = cv2.imread("/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test/happy/2705.jpg_rotation_1.jpg")
testImage = cv2.resize(testImage, (CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]))

HFModel.predict(tf.expand_dims(testImage, axis=0))

In [ ]:
HFModel.summary()

In [9]:
HFModel.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss=lossFunction,
    metrics=METRICS
)

In [10]:
class logConfusionMatrix(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        y_true = []
        y_pred = []
        for x, y in valDataset:
            y_true.extend(np.argmax(y, axis=1))
            y_pred.extend(np.argmax(self.model.predict(x), axis=1))
        wandb.log({"Confusion Matrix": wandb.plot.confusion_matrix(y_true=y_true,
                                                                    preds=y_pred,
                                                                    class_names=CONFIGURATION["CLASS_NAMES"])})

In [11]:
class logPredictionTable(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        columns = ['Image', 'Label', 'Prediction']
        table = wandb.Table(columns=columns)
        for batch in valDataset.take(1):
            for image, label in zip(*batch):
                im = wandb.Image(image)
                y_pred= np.argmax(self.model.predict(tf.expand_dims(image, axis=0)), axis=1)[0]
                row = [im, CONFIGURATION['CLASS_NAMES'][np.argmax(label)], CONFIGURATION['CLASS_NAMES'][y_pred]]
                table.add_data(*row)
        wandb.log({"Predictions": table})

In [ ]:
history = HFModel.fit(trainDataset,
                  validation_data = valDataset, 
                  epochs=CONFIGURATION["N_EPOCHS"],
                  verbose=1,
                  callbacks = [WandbMetricsLogger(), WandbModelCheckpoint("Models/EmotionDetectionViT.keras"), logConfusionMatrix(), logPredictionTable()]
)

In [ ]:
HFModel.evaluate(testDataset)

In [ ]:
wandb.run

In [ ]:
wandb.finish()